<a href="https://colab.research.google.com/github/Bugnico/Modelos1PruebasSaberPro/blob/main/03_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle
!chmod 600 ./kaggle.json


usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...
kaggle: error: the following arguments are required: command


In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.11GB/s]


In [ ]:
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:


import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:


dtr = pd.read_csv("train.csv")  # Dataset de entrenamiento (con RENDIMIENTO_GLOBAL)
dts = pd.read_csv("test.csv")   # Dataset de test (sin RENDIMIENTO_GLOBAL)

submission_example = pd.read_csv('submission_example.csv')
submission_example

,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo
...,...,...
296781,496981,alto
296782,209415,medio-alto
296783,239074,medio-alto
296784,963852,medio-bajo


In [ ]:
def categorize_program(programa):
    # Verificamos cada categoría en orden
    if programa.lower().startswith('inge') or programa.lower().startswith('bioingenier'):
        return 'INGENIERIA'
    elif programa.lower().startswith('derec') or programa.lower().startswith('jurisprudencia'):
        return 'DERECHO'
    elif programa.lower().startswith('educ') or programa.lower().startswith('pedag') or programa.lower().startswith('deporte') or programa.lower().startswith('español'):
        return 'EDUCACION'
    elif programa.lower().startswith('medic') or programa.lower().startswith('odont') or programa.lower().startswith('enferme') or programa.lower().startswith('psic') or programa.lower().startswith('instrumentacion') or programa.lower().startswith('nutrici') or programa.lower().startswith('terap'):
        return 'SALUD'
    elif programa.lower().startswith('adminis') or programa.lower().startswith('negoc') or programa.lower().startswith('banca') or programa.lower().startswith('comercio') or programa.lower().startswith('mercade'):
        return 'ADMINISTRACION'
    elif programa.lower().startswith('antropolo') or programa.lower().startswith('estudios') or programa.lower().startswith('filolo') or programa.lower().startswith('filoso') or programa.lower().startswith('sociolog') or programa.lower().startswith('teolog') or programa.lower().startswith('historia') or programa.lower().startswith('trabajo') or programa.lower().startswith('traduccion'):
        return 'HUMANAS'
    elif programa.lower().startswith('arte') or programa.lower().startswith('cine') or programa.lower().startswith('foto') or programa.lower().startswith('musica') or programa.lower().startswith('animaci') or programa.lower().startswith('bellas') or programa.lower().startswith('danza'):
        return 'ARTES'
    elif programa.lower().startswith('astronomia') or programa.lower().startswith('bacteriologia') or programa.lower().startswith('biolog') or programa.lower().startswith('ciencia') or programa.lower().startswith('agronom') or programa.lower().startswith('arqueologia') or programa.lower().startswith('geo') or programa.lower().startswith('zootecnia') or programa.lower().startswith('teolog'):
        return 'CIENCIAS'
    elif programa.lower().startswith('comunicaci') or programa.lower().startswith('direccion') or programa.lower().startswith('marketing') or programa.lower().startswith('periodismo') or programa.lower().startswith('publicidad'):
        return 'COMUNICACIONES'
    elif programa.lower().startswith('construcc'):
        return 'CONSTRUCCIONES'
    elif programa.lower().startswith('contadur') or programa.lower().startswith('econom') or programa.lower().startswith('finanz') or programa.lower().startswith('relaciones'):
        return 'ECONOMIA'
    elif programa.lower().startswith('dise') or programa.lower().startswith('diseño') :
        return 'DISEÑO'
    elif programa.lower().startswith('fisica') or programa.lower().startswith('geografia') or programa.lower().startswith('matem') or programa.lower().startswith('microbiolo') or programa.lower().startswith('quimica') or programa.lower().startswith('ecologia') or programa.lower().startswith('estad') or programa.lower().startswith('pedag') or programa.lower().startswith('bioquimica'):
        return 'EXACTAS'
    elif programa.lower().startswith('licenciatura') or programa.lower().startswith('estudios'):
        return 'LICENCIATURAS'
    elif programa.lower().startswith('profesional'):
        return 'PROFESIONAL'
    elif programa.lower().startswith('gerencia') or programa.lower().startswith('gesti') :
        return 'GESTION'
    else:
        return 'OTROS'

def categorize_departament(departament):
      if departament.lower().startswith('atlantico') or departament.lower().startswith('bolivar') or departament.lower().startswith('cordoba') or departament.lower().startswith('cesar') or departament.lower().startswith('la guajira') or departament.lower().startswith('magdalena') or departament.lower().startswith('sucre') or departament.lower().startswith('san andres'):
        return 'CARIBE'
      elif departament.lower().startswith('antioquia') or departament.lower().startswith('bogota') or departament.lower().startswith('boyaca') or departament.lower().startswith('cundinamarca') or departament.lower().startswith('risaralda') or departament.lower().startswith('tolima') or departament.lower().startswith('caldas') or departament.lower().startswith('cauca') or departament.lower().startswith('huila') or departament.lower().startswith('huila') or departament.lower().startswith('norte santander') or departament.lower().startswith('quindio') or departament.lower().startswith('santander'):
        return 'ANDINA'
      elif departament.lower().startswith('choco') or departament.lower().startswith('nariño') or departament.lower().startswith('valle'):
        return 'PACIFICA'
      elif departament.lower().startswith('arauca') or departament.lower().startswith('casanare') or departament.lower().startswith('meta'):
        return 'ORINOQUIA'
      elif departament.lower().startswith('amazonas') or departament.lower().startswith('caqueta') or departament.lower().startswith('guaviare') or departament.lower().startswith('vaupes') or departament.lower().startswith('putumayo'):
        return 'AMAZONICA'
      else:
        return 'OTRO_DEPARTAMENTO'

def limpiar_y_preprocesar_datos(d):

    d = d.drop(['FAMI_TIENEINTERNET.1', 'ESTU_PRIVADO_LIBERTAD'], axis=1)

    mapeo = {'Si': 1, 'S': 1, 'No': 0, 'N': 0}
    target_columns = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENECOMPUTADOR','ESTU_PAGOMATRICULAPROPIO']
    for columna in target_columns:
        d[columna] = d[columna].map(mapeo)

    for columna in target_columns:
        proporciones = d[columna].value_counts(normalize=True)
        prop_1 = proporciones.get(1, 0)
        prop_0 = proporciones.get(0, 0)
        d[columna].fillna(np.random.choice([0, 1], p=[prop_0, prop_1]), inplace=True)
    d['ESTU_PAGOMATRICULAPROPIO'].fillna(0, inplace=True)

    mapeo_estrato = {'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}
    d['FAMI_ESTRATOVIVIENDA'] = d['FAMI_ESTRATOVIVIENDA'].map(mapeo_estrato)
    media_estrato = d['FAMI_ESTRATOVIVIENDA'].mean()
    d['FAMI_ESTRATOVIVIENDA'].fillna(media_estrato, inplace=True)
    d['FAMI_ESTRATOVIVIENDA'] = d['FAMI_ESTRATOVIVIENDA'].astype(int)

    mapeo_valores_matricula = {'Menos de 500 mil': 0, 'Entre 500 mil y menos de 1 millón': 1, 'Entre 1 millón y menos de 2.5 millones': 2,
                                'Entre 2.5 millones y menos de 4 millones': 3, 'Entre 4 millones y menos de 5.5 millones': 4,
                                'Entre 5.5 millones y menos de 7 millones': 5, 'Más de 7 millones': 6, 'No pagó matrícula': 7}

    mapeo_valores_horas_trabajo = {'0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2, 'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4}

    d['ESTU_VALORMATRICULAUNIVERSIDAD'] = d['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapeo_valores_matricula)
    d['ESTU_HORASSEMANATRABAJA'] = d['ESTU_HORASSEMANATRABAJA'].map(mapeo_valores_horas_trabajo)

    media_valormatricula = d['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()
    d['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(media_valormatricula, inplace=True)

    valores_horas_trabajo = d['ESTU_HORASSEMANATRABAJA'].dropna().unique()
    num_valores_faltantes = d['ESTU_HORASSEMANATRABAJA'].isnull().sum()
    d.loc[d['ESTU_HORASSEMANATRABAJA'].isnull(), 'ESTU_HORASSEMANATRABAJA'] = np.random.choice(valores_horas_trabajo, num_valores_faltantes)

    moda_padre = d['FAMI_EDUCACIONPADRE'].mode()[0]
    moda_madre = d['FAMI_EDUCACIONMADRE'].mode()[0]

    d['FAMI_EDUCACIONPADRE'].fillna(moda_padre, inplace=True)
    d['FAMI_EDUCACIONMADRE'].fillna(moda_madre, inplace=True)


    d['ESTU_PRGM_ACADEMICO'] = d['ESTU_PRGM_ACADEMICO'].apply(lambda x: categorize_program(x))
    d['ESTU_PRGM_DEPARTAMENTO'] = d['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: categorize_departament(x))

    d = pd.get_dummies(d, columns=['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PRGM_ACADEMICO','ESTU_PRGM_DEPARTAMENTO'], prefix=['PADRE_EDUCACION', 'MADRE_EDUCACION','ESTU_PRGM_ACADEMICO','ESTU_PRGM_DEPARTAMENTO'])
    d.replace({True: 1, False: 0}, inplace=True)

    return d

In [ ]:

lentr = len(dtr)
dtr.shape, dts.shape


((692500, 21), (296786, 20))

In [ ]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = limpiar_y_preprocesar_datos(all_data)

Xtrk, ytrk = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xtsk  = all_data.iloc[lentr:].values

print (Xtrk.shape, ytrk.shape)
print (Xtsk.shape)

/tmp/ipython-input-7-1608646537.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  d[columna].fillna(np.random.choice([0, 1], p=[prop_0, prop_1]), inplace=True)
/tmp/ipython-input-7-1608646537.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].m

(692500, 61) (692500,)
(296786, 61)


In [ ]:


n = int(len(Xtrk)*0.5)
n

346250

In [ ]:

idxs = np.random.permutation(len(Xtrk))
idxs_trm = idxs[:n]
idxs_tsm = idxs[n:]

Xtrm = Xtrk[idxs_trm]
ytrm = ytrk[idxs_trm]

Xtsm = Xtrk[idxs_tsm]
ytsm = ytrk[idxs_tsm]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Modelo base: Regresión Logística
logreg = LogisticRegression(
    solver='lbfgs',
    max_iter=1000,
    random_state=42
)

# Entrenamiento
logreg.fit(Xtrm, ytrm)

# Predicción
ypred_logreg = logreg.predict(Xtsm)

# Evaluación
print("Accuracy:", accuracy_score(ytsm, ypred_logreg))
print("\nClassification Report:\n", classification_report(ytsm, ypred_logreg))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.366457761732852

Classification Report:
               precision    recall  f1-score   support

        alto       0.43      0.57      0.49     87642
        bajo       0.37      0.54      0.44     86616
  medio-alto       0.28      0.19      0.23     85821
  medio-bajo       0.30      0.16      0.20     86171

    accuracy                           0.37    346250
   macro avg       0.35      0.37      0.34    346250
weighted avg       0.35      0.37      0.34    346250

